# Metrics

Here are common metrics that have been designed or adapted specifically for recommendation systems.

In [86]:
import pandas as pd
from IPython.display import HTML
from sklearn.datasets import make_blobs

## Example task

The following cell has generated an assignment that we will use as an example. It is generated in the format `<user/item> <-> rating`.

In [94]:
r_width = 10
r_height = 30

R, c = make_blobs(
    n_samples=r_height,
    n_features=r_width,
    centers=3,
    random_state=10
)
R = np.round((R-R.min())*10/(R.max()-R.min())).astype(int)

# genrating combinations of object/item to be empty
combination_counts = 20
nan_combinations = np.concatenate(
    [
        np.random.randint(0, R.shape[0], [combination_counts,1]),
        np.random.randint(0, R.shape[1], [combination_counts,1])
    ],
    axis=1
)

# replacing some combinations with emptys
R = R.astype("float32")
for comb in nan_combinations:
    R[tuple(comb)] = np.NaN

R_frame = pd.Series(
    R.ravel(),
    index = pd.MultiIndex.from_tuples(
            [
                (j,i) 
                for j in np.arange(R.shape[1]) 
                for i in np.arange(R.shape[0])
            ],
            names = ["object", "item"]
    ),
    name = "rating"
).reset_index()

R_frame.sample(10)

,object,item,rating
173,5,23,7.0
57,1,27,5.0
271,9,1,0.0
92,3,2,5.0
85,2,25,4.0
77,2,17,5.0
190,6,10,7.0
253,8,13,5.0
26,0,26,6.0
5,0,5,6.0


## precision@k

array([ 7.,  9.,  0.,  6.,  8.,  6.,  7.,  3.,  9.,  8.,  6., 10.,  1.,
        4.,  9.,  6., nan,  3.,  8.,  7.,  5.,  2.,  4.,  6.,  5.,  4.,
        6., nan,  6.,  6.,  6.,  9.,  1., nan,  8., nan,  8.,  3., nan,
        7.,  5.,  2.,  4.,  7., nan,  5.,  6.,  5.,  7.,  6.,  6.,  1.,
       nan,  7.,  4.,  5.,  7., nan,  8.,  6.,  7.,  9.,  1.,  6., nan,
        6.,  7.,  3.,  9.,  8.,  5.,  2.,  4.,  6.,  5.,  5.,  6.,  5.,
        6.,  6., nan,  2.,  4.,  6.,  4.,  4.,  6.,  5.,  7.,  6.,  6.,
        1.,  5.,  7.,  4.,  4.,  6.,  5.,  7.,  7., nan, nan,  1.,  5.,
        8.,  7.,  7.,  3.,  8.,  6., nan,  9.,  1.,  5.,  7.,  6.,  8.,
        3.,  9.,  7.,  9.,  1.,  6.,  7.,  5.,  2.,  3.,  7., nan,  1.,
        6.,  2.,  4.,  6., nan,  4.,  7.,  5.,  7.,  6.,  8.,  1.,  6.,
        8.,  5.,  2.,  2.,  7.,  3.,  1.,  8.,  1.,  6.,  6.,  6.,  3.,
        2.,  7.,  2.,  2.,  7.,  1.,  7.,  7.,  5.,  2.,  3.,  7.,  2.,
        1.,  5.,  1.,  4.,  7.,  4.,  5.,  6.,  4.,  6.,  6.,  6

In [11]:
R

array([[ 7.,  9.,  0.,  6.,  8.,  6.,  7.,  3.,  9.,  8.],
       [ 6., 10.,  1.,  4.,  9.,  6., nan,  3.,  8.,  7.],
       [ 5.,  2.,  4.,  6.,  5.,  4.,  6., nan,  6.,  6.],
       [ 6.,  9.,  1., nan,  8., nan,  8.,  3., nan,  7.],
       [ 5.,  2.,  4.,  7., nan,  5.,  6.,  5.,  7.,  6.],
       [ 6.,  1., nan,  7.,  4.,  5.,  7., nan,  8.,  6.],
       [ 7.,  9.,  1.,  6., nan,  6.,  7.,  3.,  9.,  8.],
       [ 5.,  2.,  4.,  6.,  5.,  5.,  6.,  5.,  6.,  6.],
       [nan,  2.,  4.,  6.,  4.,  4.,  6.,  5.,  7.,  6.],
       [ 6.,  1.,  5.,  7.,  4.,  4.,  6.,  5.,  7.,  7.],
       [nan, nan,  1.,  5.,  8.,  7.,  7.,  3.,  8.,  6.],
       [nan,  9.,  1.,  5.,  7.,  6.,  8.,  3.,  9.,  7.],
       [ 9.,  1.,  6.,  7.,  5.,  2.,  3.,  7., nan,  1.],
       [ 6.,  2.,  4.,  6., nan,  4.,  7.,  5.,  7.,  6.],
       [ 8.,  1.,  6.,  8.,  5.,  2.,  2.,  7.,  3.,  1.],
       [ 8.,  1.,  6.,  6.,  6.,  3.,  2.,  7.,  2.,  2.],
       [ 7.,  1.,  7.,  7.,  5.,  2.,  3.,  7.,  2.,  1.